In [28]:
import numpy as np
import pandas as pd
from io import BytesIO
from pymongo import MongoClient
import boto3
from io import StringIO 
from pathlib import Path

## IMPORTANDO ARQUIVOS CSV

In [36]:
df = pd.read_csv('municipios-estados/csv/estados.csv')
df_municipios = pd.read_csv('municipios-estados/csv/municipios.csv')

## TRATANDO DUPLICIDADES

In [27]:
df_uf = df['uf'].drop_duplicates()
df_uf

0     RO
1     AC
2     AM
3     RR
4     PA
5     AP
6     TO
7     MA
8     PI
9     CE
10    RN
11    PB
12    PE
13    AL
14    SE
15    BA
16    MG
17    ES
18    RJ
19    SP
20    PR
21    SC
22    RS
23    MS
24    MT
25    GO
26    DF
Name: uf, dtype: object

## CRIANDO PASTAS DE CADA UF ENCONTRADO NO ARQUIVO

In [54]:
caminho_base = 'municipios-estados/tarefa-aula-06'

for valor in df_uf:
    nome_da_linha = str(valor)
    caminho_completo = Path(caminho_base) / nome_da_linha
    caminho_completo.mkdir(parents=True, exist_ok=True)

In [55]:
df_municipios

,codigo_ibge,nome,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario
0,5200050,Abadia de Goiás,-16.75730,-49.4412,0,52,1050,62,America/Sao_Paulo
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31,4001,34,America/Sao_Paulo
2,5200100,Abadiânia,-16.19700,-48.7057,0,52,9201,62,America/Sao_Paulo
3,3100203,Abaeté,-19.15510,-45.4444,0,31,4003,37,America/Sao_Paulo
4,1500107,Abaetetuba,-1.72183,-48.8788,0,15,401,91,America/Sao_Paulo
...,...,...,...,...,...,...,...,...,...
5565,2933604,Xique-Xique,-10.82300,-42.7245,0,29,3971,74,America/Sao_Paulo
5566,2517407,Zabelê,-8.07901,-37.1057,0,25,542,83,America/Sao_Paulo
5567,3557154,Zacarias,-21.05060,-50.0552,0,35,2973,18,America/Sao_Paulo
5568,2114007,Zé Doca,-3.27014,-45.6553,0,21,1287,98,America/Sao_Paulo


In [56]:
df

,codigo_uf,uf,nome,latitude,longitude,regiao
0,11,RO,Rondônia,-10.83,-63.34,Norte
1,12,AC,Acre,-8.77,-70.55,Norte
2,13,AM,Amazonas,-3.47,-65.10,Norte
3,14,RR,Roraima,1.99,-61.33,Norte
4,15,PA,Pará,-3.79,-52.48,Norte
5,16,AP,Amapá,1.41,-51.77,Norte
6,17,TO,Tocantins,-9.46,-48.26,Norte
7,21,MA,Maranhão,-5.42,-45.44,Nordeste
8,22,PI,Piauí,-6.60,-42.28,Nordeste
9,23,CE,Ceará,-5.20,-39.53,Nordeste


## BUSCANDO UF CORRESPONDENTE A CIDADE

In [57]:
df_consolidado = pd.merge(df_municipios,df[['uf', 'codigo_uf']], left_on='codigo_uf', right_on='codigo_uf', how='left')
df_consolidado = df_consolidado.rename(columns={'nome': 'cidade'})
df_consolidado

,codigo_ibge,cidade,latitude,longitude,capital,codigo_uf,siafi_id,ddd,fuso_horario,uf
0,5200050,Abadia de Goiás,-16.75730,-49.4412,0,52,1050,62,America/Sao_Paulo,GO
1,3100104,Abadia dos Dourados,-18.48310,-47.3916,0,31,4001,34,America/Sao_Paulo,MG
2,5200100,Abadiânia,-16.19700,-48.7057,0,52,9201,62,America/Sao_Paulo,GO
3,3100203,Abaeté,-19.15510,-45.4444,0,31,4003,37,America/Sao_Paulo,MG
4,1500107,Abaetetuba,-1.72183,-48.8788,0,15,401,91,America/Sao_Paulo,PA
...,...,...,...,...,...,...,...,...,...,...
5565,2933604,Xique-Xique,-10.82300,-42.7245,0,29,3971,74,America/Sao_Paulo,BA
5566,2517407,Zabelê,-8.07901,-37.1057,0,25,542,83,America/Sao_Paulo,PB
5567,3557154,Zacarias,-21.05060,-50.0552,0,35,2973,18,America/Sao_Paulo,SP
5568,2114007,Zé Doca,-3.27014,-45.6553,0,21,1287,98,America/Sao_Paulo,MA


## EXPORTANDO ARQUIVO CSV POR CIDADE EORDENADO POR UF

In [58]:
df_cidades = df_consolidado[['cidade','uf']].sort_values(by='uf').reset_index(drop=True)
df_cidades.to_csv('municipios-estados/tarefa-aula-06/cidades.csv')

In [52]:
caminho_base = 'municipios-estados/'

for uf, grupo in df_cidades.groupby('uf'):
    caminho_pasta = Path(caminho_base) / uf
    caminho_pasta.mkdir(parents=True, exist_ok=True)
    caminho_arquivo_csv = caminho_pasta / 'cidade.csv'
    grupo[['cidade','uf']].to_csv(caminho_arquivo_csv, index=False, header=True)

print(f'Arquivos CSV gerados')

Arquivos CSV gerados


In [2]:
client = boto3.client('s3', 
    endpoint_url='http://awari-minio-nginx:9000',
    aws_access_key_id='EVrYpCgbPdv0KWQV',
    aws_secret_access_key='Hlli13j51CZLRdGlFkO2RZzcIZ1hnOvt',
    aws_session_token=None,
    config=boto3.session.Config(signature_version='s3v4'),
    verify=False,
    region_name='sa-east-1'
)

In [64]:
csv_buffer = StringIO()
df_cidades.to_csv(csv_buffer)
client.put_object(Body=csv_buffer.getvalue(), Bucket='tarefa-aula-06', Key="cidades/cidades.csv")

{'ResponseMetadata': {'RequestId': '179BEC8FE26D698F',
  'HostId': 'e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'nginx/1.19.2',
   'date': 'Tue, 28 Nov 2023 23:16:27 GMT',
   'content-length': '0',
   'connection': 'keep-alive',
   'accept-ranges': 'bytes',
   'content-security-policy': 'block-all-mixed-content',
   'etag': '"549d575e3d3f857d24cb73bfeb6bc27e"',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'vary': 'Origin, Accept-Encoding',
   'x-amz-bucket-region': 'sa-east-1',
   'x-amz-id-2': 'e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855',
   'x-amz-request-id': '179BEC8FE26D698F',
   'x-content-type-options': 'nosniff',
   'x-xss-protection': '1; mode=block'},
  'RetryAttempts': 0},
 'ETag': '"549d575e3d3f857d24cb73bfeb6bc27e"'}